In [45]:
from pystac_client import Client
from odc.stac import load
import odc.geo

In [85]:
# client = Client.open("https://catalogue.dataspace.copernicus.eu/stac") 
client = Client.open("https://earth-search.aws.element84.com/v1") 
search = client.search(
    max_items=10,
    collections=['sentinel-2-l2a'],
    bbox=[-8,42.95,-7.95,43],
    datetime='2022'
)
print(f"{search.matched()} items found")


147 items found


In [41]:
for item in search.items():
    print(item.id)

S2B_29TNH_20221230_0_L2A
S2B_29TPH_20221230_0_L2A
S2A_29TNH_20221228_0_L2A
S2A_29TPH_20221228_0_L2A
S2A_29TNH_20221225_0_L2A
S2A_29TPH_20221225_0_L2A
S2B_29TNH_20221223_0_L2A
S2B_29TPH_20221223_0_L2A
S2B_29TNH_20221220_0_L2A
S2B_29TPH_20221220_0_L2A


In [86]:
item_collection = search.item_collection()
#item_collection.save_object('sentinel2-items.json')

In [84]:
import rasterio
for item in item_collection:
    print('Nieri')
    print(item.properties)
    print(item.properties['eo:cloud_cover'])
    break

Nieri
{'created': '2022-12-30T18:25:41.937Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 98.552608, 'proj:epsg': 32629, 'mgrs:utm_zone': 29, 'mgrs:latitude_band': 'T', 'mgrs:grid_square': 'NH', 'grid:code': 'MGRS-29TNH', 'view:sun_azimuth': 163.543584233916, 'view:sun_elevation': 22.248166700303997, 's2:degraded_msi_data_percentage': 0.0127, 's2:nodata_pixel_percentage': 22.191423, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0.008272, 's2:cloud_shadow_percentage': 1.4241, 's2:vegetation_percentage': 0, 's2:not_vegetated_percentage': 0.002878, 's2:water_percentage': 0.010396, 's2:unclassified_percentage': 0.001744, 's2:medium_proba_clouds_percentage': 21.125785, 's2:high_proba_clouds_percentage': 71.217191, 's2:thin_cirrus_percentage': 6.209628, 's2:snow_ice_percentage': 0, 's2:product_type': 'S2MSI2A', 's2:processing_baseline': '05.09', 's2:product_uri': 'S2B_MSIL2A_20221230T112359_N0509_R037_T29T

In [53]:
bands_sentinel2 = ['blue', 'swir16']


In [87]:
xx = odc.stac.load(

    item_collection,

    bands= bands_sentinel2,

    crs="EPSG:25830",
    resolution=10

)

In [89]:
xx.median(dim='time',skipna=True)

: 

In [76]:
xx.spatial_ref.GeoTransform


'5600 400 0 4817600 0 -400'

In [62]:
ndvi = (xx.red - xx.nir) / (xx.red + xx.nir)

ndvi_p10 = ndvi.quantile(0.1,dim='time',skipna = True)
ndvi_p90 = ndvi.quantile(0.9,dim='time',skipna = True)
ndvi_median = ndvi.median(dim='time', skipna = True)

ndvi_median.fillna(255)

/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


<xarray.DataArray (y: 295, x: 295)> Size: 696kB
array([[255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       ...,
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.],
       [255., 255., 255., ..., 255., 255., 255.]])
Coordinates:
  * y            (y) float64 2kB 4.817e+06 4.817e+06 ... 4.7e+06 4.7e+06
  * x            (x) float64 2kB 5.8e+03 6.2e+03 6.6e+03 ... 1.23e+05 1.234e+05
    spatial_ref  int32 4B 25830

In [72]:
import xarray as xr
import rioxarray as rio

# median_blue = xx.blue.median(dim='time',skipna=True)
# median_red = xx.red.median(dim='time',skipna=True)
# median_green = xx.green.median(dim='time',skipna=True)
# median_nir = xx.nir.median(dim='time',skipna=True)
# median_swir16 = xx.swir16.median(dim='time',skipna=True)
# median_swir22 = xx.swir22.median(dim='time',skipna=True)

# median_blue

# xx

xx.median(dim='time',skipna=True)


# dataset = xr.merge([median_blue,median_red,median_green])
# dataset.rio.to_raster("test_multiband.tif")

<xarray.Dataset> Size: 4MB
Dimensions:      (y: 295, x: 295)
Coordinates:
  * y            (y) float64 2kB 4.817e+06 4.817e+06 ... 4.7e+06 4.7e+06
  * x            (x) float64 2kB 5.8e+03 6.2e+03 6.6e+03 ... 1.23e+05 1.234e+05
    spatial_ref  int32 4B 25830
Data variables:
    blue         (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    green        (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    red          (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    nir          (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    swir16       (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    swir22       (y, x) float64 696kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0